# Libraries



In [ ]:
import tensorflow as tf
import keras 
import PIL
from PIL import Image
import cv2
import numpy as np
import os

from tqdm import tqdm
from keras import layers
from keras.models import Model
from keras.utils.vis_utils import plot_model
from keras import optimizers
import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

In [ ]:
print(tf.__version__)

# Loading


**Loading sequences from directories**

In [ ]:
os.mkdir('./training/')


In [ ]:
os.mkdir('./test/')

In [ ]:
import csv
def get_t2_start(file_name):
    file_name=file_name+'_phases.csv'
    BASE_DIR="../input/130embrayovids/embryo_dataset_annotations/embryo_dataset_annotations"
    file_path=os.path.join(BASE_DIR,file_name)
    with open(file_path,'r') as f:
       csv_reader=csv.reader(f) 
       for row in csv_reader:
           if row[0]=='t2':
               return int(row[1])
    return 0

In [ ]:
def Load_labels(t2_time,j,length,TARGET_PATH):
  array=np.ones((length,1))
  array2=np.zeros((length,1))
  for i in range(t2_time):
      array[i]=0
      array2[i]=1
  array=np.expand_dims(array, axis=0)
  array2=np.expand_dims(array2, axis=0)
  category=np.stack([array2,array],axis=0)
  #category=np.expand_dims(array, axis=2)
  np.save(TARGET_PATH+'/category_{}.npy'.format(j),category)

In [ ]:
def sort_by_num(name):
  test=name
  test=test[32:]
  test=test[:-5]
  try:
    return(int('0'+test))
  except:
    name=name[33:]
    name=name[:-5]
    return(int('0'+name))

In [ ]:
from PIL import ImageOps

In [ ]:
img = Image.open('../input/130embrayovids/preprocessed data(130 sequence)/preprocessed data(130 sequence)/CA704_EMB8_DISCARD/D2017.07.07_S1982_I132_WELL8_RUN102.jpeg').convert('L')
img

In [ ]:
img=PIL.ImageOps.autocontrast(PIL.ImageOps.equalize(img))
img

In [ ]:
img = Image.open('../input/130embrayovids/preprocessed data(130 sequence)/preprocessed data(130 sequence)/AM33-2/D2014.01.21_S0993_I132_WELL2_RUN100.jpeg').convert('L')
img

In [ ]:
img=PIL.ImageOps.autocontrast(PIL.ImageOps.equalize(img))
img


In [ ]:
#with PIL
def Load_seq(BASE_DIRECTORY,TARGET_PATH,length,sorting_func):
  i=0
  for directory in os.listdir(BASE_DIRECTORY):
    filenames=sorted(os.listdir(os.path.join(BASE_DIRECTORY,directory)),key=sorting_func)
    stack=[]
    for filename in tqdm(filenames):   # tqdm is for displaying progress bar 
      img = Image.open(os.path.join(os.path.join(BASE_DIRECTORY,directory),filename)).convert('L')#test grayscale
      img=PIL.ImageOps.autocontrast(PIL.ImageOps.equalize(img)) #magic
      numpydata = (np.asarray(img))/255 #normalizing data
      numpydata = np.expand_dims(numpydata,axis=-1)
      #numpydata = np.moveaxis(numpydata, -1, 0)
      stack.append(numpydata)
    output=np.stack(stack, axis=0)
    np.save(TARGET_PATH+'/scene_{}.npy'.format(i),output)
    print(get_t2_start(directory)-70)
    Load_labels(get_t2_start(directory)-70,i,length,TARGET_PATH)
    i=i+1


In [ ]:
TARGET_PATH='./training/'
BASE_DIRECTORY='../input/130embrayovids/preprocessed data(130 sequence)/preprocessed data(130 sequence)'
%time Load_seq(BASE_DIRECTORY,TARGET_PATH,50,sort_by_num)  #1min 17s

In [ ]:
TARGET_PATH='./test/'
BASE_DIRECTORY='../input/gomez-30-test/gomez30_test'
%time Load_seq(BASE_DIRECTORY,TARGET_PATH,50,sort_by_num)  #1min 17s

#Create Annotated Video


# Data Augmentation

In [ ]:
def DataAugmentation(seq,angles,fliph=True,flipv=True):
  sequence = np.load(r"./training/scene_{}.npy".format(seq))
  l=[]
  if fliph:
    l.append(flip_h(sequence))
  if flipv:
    l.append(flip_v(sequence))
  for a in angles:
    l.append(rotate(sequence,a))
  return l

In [ ]:
def flip_h(sequence):
  stack=[]
  for i in range(0,50):
        im=sequence[i].reshape((250,250))
        stack.append(np.expand_dims(cv2.flip(im,1),axis=-1))  
  output=np.stack(stack, axis=0)
  return output 

In [ ]:
def flip_v(sequence):
  stack=[]
  for i in range(0,50):
        im=sequence[i].reshape((250,250))
        stack.append(np.expand_dims(cv2.flip(im, 0),axis=-1))  
  output=np.stack(stack, axis=0)
  return output

In [ ]:
def rotate(sequence,angle):
  stack=[]
  for i in range(0,50):
    im=sequence[i].reshape((250,250))
    M = cv2.getRotationMatrix2D((250/2, 250/2),angle, 1.0)
    rotated = cv2.warpAffine(im, M, im.shape)
    rotated=np.expand_dims(im,axis=-1)
    stack.append(rotated)
  output=np.stack(stack, axis=0)
  return output

In [ ]:
#%timeit rotate(0,45)

In [ ]:
#%timeit DataAugmentation(0,[45,90,135,180,225])
#DataAugmentation(0,[45,90,135,180,225])[1].shape

# training

## Model

In [ ]:

def My_ConvLSTM_Model(frames, pixels_x, pixels_y ,channels, categories):
  
    trailer_input  = layers.Input(shape=(frames, pixels_x, pixels_y,channels)
                    , name='trailer_input')
    
    first_ConvLSTM = layers.ConvLSTM2D(filters=40, kernel_size=(3, 3)
                       , data_format='channels_last'
                       , recurrent_activation='hard_sigmoid' #hard sigmoid will allow the prediction to change fast enough (when testing simple sigmoid the prediction cant change toward t2)
                       , activation='tanh'
                       , padding='same'
                        ,kernel_regularizer=keras.regularizers.L2(0.01)               
                       , return_sequences=True)(trailer_input)
    first_BatchNormalization = layers.BatchNormalization()(first_ConvLSTM)
    first_Pooling = layers.MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last')(first_BatchNormalization)
    
    second_ConvLSTM = layers.ConvLSTM2D(filters=20, kernel_size=(3, 3)
                        , data_format='channels_last'
                        , dropout=0.1
                        , padding='same'
                        ,kernel_regularizer=keras.regularizers.L2(0.01)              
                        , return_sequences=True)(first_Pooling)
    second_BatchNormalization = layers.BatchNormalization()(second_ConvLSTM)
    second_Pooling = layers.MaxPooling3D(pool_size=(1, 3, 3), padding='same', data_format='channels_last')(second_BatchNormalization)
    
    outputs = [branch(second_Pooling, 'cat_{}'.format(category)) for category in categories]
    
    seq = keras.models.Model(inputs=trailer_input, outputs=outputs, name='Model')
    
    return seq

def branch(last_convlstm_layer, name):
  
    branch_ConvLSTM = layers.ConvLSTM2D(filters=12, kernel_size=(3, 3)
                        , data_format='channels_last'
                        , stateful = False
                        , kernel_initializer='random_uniform'
                        ,kernel_regularizer=keras.regularizers.L2(0.01)
                        , padding='same', return_sequences=True)(last_convlstm_layer)
    branch_Pooling = layers.MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last')(branch_ConvLSTM)
    flat_layer = layers.TimeDistributed(layers.Flatten())(branch_Pooling)
    
    first_Dense = layers.TimeDistributed(layers.Dense(256,kernel_regularizer=keras.regularizers.L2(0.01)))(flat_layer)
    second_Dense = layers.TimeDistributed(layers.Dense(32,kernel_regularizer=keras.regularizers.L2(0.01)))(first_Dense)
    dropout=layers.TimeDistributed(layers.Dropout(0.2))(second_Dense)
    target = layers.TimeDistributed(layers.Dense(1,activation='sigmoid'), name=name)(dropout)
    
    return target
    

## fit Generator

In [ ]:
def generate_arrays(available_ids):
    from random import shuffle
	
    while True:
        
        shuffle(available_ids)
        for i in available_ids:
            
            scene = np.load('./training/scene_{}.npy'.format(i))
            category = np.load('./training/category_{}.npy'.format(i))
            yield (np.array([scene]), [category[0],category[1]])

In [ ]:
def generate_arrays_augmented(available_ids):
    from random import shuffle
	
    while True:
        
        shuffle(available_ids)
        for i in available_ids:
            
            scene = np.load('./training/scene_{}.npy'.format(i))
            category = np.load('./training/category_{}.npy'.format(i))
            for seq in DataAugmentation(i,[]):
              yield(np.array([seq]),[category[0],category[1]])
            yield (np.array([scene]), [category[0],category[1]])

## Callback

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5)
checkpoint = keras.callbacks.ModelCheckpoint('./checkpoint', monitor='val_loss',save_best_only=True, mode='min')

## Fitting the data

In [ ]:
available_ids = [i for i in range(0, 130)]

from random import shuffle
shuffle(available_ids)

train_ids = available_ids[0:110]
val_ids = available_ids[110:129]

# fit the model
model=My_ConvLSTM_Model(50,250,250,1,['t1','t2'])
model.compile(loss=[keras.losses.binary_crossentropy,keras.losses.binary_crossentropy], optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),metrics=['accuracy',tf.keras.metrics.Recall()])
history = model.fit(
    generate_arrays_augmented(train_ids)
    , steps_per_epoch = len(train_ids)
    , validation_data = generate_arrays(val_ids)
    , validation_steps = len(val_ids)
    , epochs = 35
    , verbose = 1
    , shuffle = False
    , initial_epoch = 0
    ,callbacks=[early_stopping,reduce_lr,checkpoint]
    )


## Visualization

In [ ]:
model.save('./latest_model_autocont_normal')

In [ ]:
plot_model(model, to_file='./model_architecture.png', show_shapes=True, show_layer_names=True)

In [ ]:
def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.savefig('./model_loss.png')
  plt.show()

In [ ]:
plot_graphs(history, 'loss')

In [ ]:
def plot_graphs_branch(history,branchs):
  for i in range(len(branchs)):
    plt.plot(history.history[branchs[i]])
    plt.plot(history.history['val_'+branchs[i]])
    plt.xlabel("Epochs")
    plt.ylabel(branchs[i])
    plt.legend([branchs[i], 'val_'+branchs[i]])
    #plt.savefig('./model_met{}.png'.format(i))
    plt.show()

In [ ]:
branchs=['cat_t1_accuracy','cat_t2_accuracy']
plot_graphs_branch(history,branchs)

In [ ]:
branchs=['cat_t1_recall','cat_t2_recall']
plot_graphs_branch(history,branchs)

## Predictions

In [ ]:
#model = keras.models.load_model('../input/autoconst-model/latest_model_autocont')

In [ ]:
def MeanEntropy(model,test):
  probs=model.predict(test)[0][0].reshape((50,))
  en1=0
  en2=0
  for i in probs:
    en1 -=((i* log(i,2))+((1-i)*log(1-i,2)))/50
  probs=model.predict(test)[1][0].reshape((50,))
  for i in probs:
    en2 -=((i* log(i,2))+((1-i)*log(1-i,2)))/50
  return en1,en2,(en1<en2)
  

In [ ]:
def get_pred(model,test):
  l1=[]
  l2=[]
  for ele in model.predict(test)[0][0]:
    if ele[0]>0.5:
      l1.append(1)
    else:
      l1.append(0)
  pred_t1=np.array(l1)
  for ele in model.predict(test)[1][0]:
    if ele[0]<0.5:
      l2.append(1)
    else:
      l2.append(0)
  pred_t2=np.array(l2)
  return pred_t1,pred_t2


In [ ]:
def accuarcy_with_intervall(model,scene,intervall,mode=1): #mode1 : min entropy , mode 2 : choose t1, mode 3 : choose t2
  test=np.load('./training/scene_{}.npy'.format(scene))
  test=np.expand_dims(test,axis=0)
  category=np.load('./training/category_{}.npy'.format(scene))
  gt=category[0].reshape(-1)
  p1,p2=get_pred(model,test)
  en1,en2,t1_is_more_certain=MeanEntropy(model,test)
  if mode==1:
    if t1_is_more_certain :
      time_predicted=np.argmax(p1<1)
    else:
      time_predicted=np.argmax(p2<1)
  elif mode==2:
      time_predicted=np.argmax(p1<1)
  elif mode==3:
      time_predicted=np.argmax(p2<1)
  else:
    print('no such mode')
  t2_time_gt=np.argmax(gt<1)
  #print('t1 is chosen: ',t1_is_more_certain)
  print('sequence num {} : predicted : '.format(scene),time_predicted,' reality : ',t2_time_gt)
  if abs(time_predicted-t2_time_gt)<(intervall+1):
    truth=1
    diff=abs(time_predicted-t2_time_gt)
    return truth,diff
  else :
    truth=0
    diff=abs(time_predicted-t2_time_gt)
    return truth,diff

In [ ]:
from math import log
def MeanEntropyInConflictedZone(model,test):
  probs1=model.predict(test)[0][0].reshape((50,))
  probs2=model.predict(test)[1][0].reshape((50,))
  p1,p2=get_pred(model,test)
  mask=~(p1==p2)
  en1=0
  en2=0
  for i in probs1[mask]:
    en1 -=((i* log(i,2))+((1-i)*log(1-i,2)))/np.count_nonzero(mask)
  
  for i in probs2[mask]:
    en2 -=((i* log(i,2))+((1-i)*log(1-i,2)))/np.count_nonzero(mask)
  return en1,en2,(en1>en2)
  

In [ ]:
def get_t2(p):
    if p.all():
        return 50
    elif p.any()==False :
        return 0
    else:
        return np.argmax(p<1)

In [ ]:
def accuarcy_with_intervall_conf(model,scene,intervall,path,mode=1): #mode1 : min entropy , mode 2 : choose t1, mode 3 : choose t2
  test=np.load(path+'/scene_{}.npy'.format(scene))
  test=np.expand_dims(test,axis=0)
  category=np.load(path+'/category_{}.npy'.format(scene))
  gt=category[0].reshape(-1)
  p1,p2=get_pred(model,test)
  if mode==1:
    en1,en2,t1_is_more_certain=MeanEntropyInConflictedZone(model,test)
    if t1_is_more_certain :  
      time_predicted=get_t2(p1)
    else:
      time_predicted=get_t2(p2)
  elif mode==2:
      time_predicted=get_t2(p1)
  elif mode==3:
      time_predicted=get_t2(p2)
  else:
    print('no such mode')
  t2_time_gt=get_t2(gt)
  #print('t1 is chosen: ',t1_is_more_certain)
  #if time_predicted==0:
   # print('sequence num {} : predicted : '.format(scene),'undetected',' reality : ',t2_time_gt)
  #else:
    #print('sequence num {} : predicted : '.format(scene),time_predicted,' reality : ',t2_time_gt)
  if abs(time_predicted-t2_time_gt)<(intervall+1):
    truth=1
    diff=abs(time_predicted-t2_time_gt)
    return truth,diff
  else :
    truth=0
    if time_predicted==0:
      diff=0
    else:
      diff=abs(time_predicted-t2_time_gt)
    return truth,diff

In [ ]:
#model1=keras.models.load_model('../input/autoconst-model/latest_model_autocont')

In [ ]:
#model2_checkpoint=keras.models.load_model('../input/aug-model-withdropout/model_aug_dropout/checkpoint')

In [ ]:
#model_test=keras.models.load_model('../input/model-normal/model_normal/latest_model_autocont_normal')

In [ ]:
from math import log

for j in range(0,6):
    cum_acc=0
    cum_diff=0
    for i in range(0,130) :
      truth,diff=accuarcy_with_intervall_conf(model,i,j,'./training/',1)
      cum_acc=cum_acc+truth
      cum_diff=cum_diff+diff
    mean_diff=cum_diff/130
    accuarcy=cum_acc/130
    print('Précision avec un intervalle de {} frames : '.format(j),accuarcy*100,'%')
    print('Différence moyenne : ',mean_diff)

In [ ]:
from math import log
for j in range(0,6):
    cum_acc=0
    cum_diff=0
    for i in range(0,30) :
      truth,diff=accuarcy_with_intervall_conf(model,i,j,'./test/',1)
      cum_acc=cum_acc+truth
      cum_diff=cum_diff+diff
    mean_diff=cum_diff/30
    accuarcy=cum_acc/30
    print('Précision avec un intervalle de {} frames : '.format(j),accuarcy*100,'%')
    print('Différence moyenne : ',mean_diff)